In [2]:
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("Datasets/OriginalData.json")

In [4]:
len(df)

34039

Need to put USAF datasets into one full dataframe, which is achieved by the following loop. "USAFDfs" will be a list of dataframes from each year spanning 1981-2004.

In [5]:
USAFDfs = []

In [6]:
q=0
n = 8
while(len(USAFDfs) <= 14 ):
  q += 1
  if (q >= 10):
    q -= 10
    n = 9
  Df = pd.read_csv("Datasets/USAF/DfEd3_19{}{}_usaf.csv".format(n,q))
  USAFDfs.append(Df)

Removing unnecessary columns:

In [10]:
for i in range(len(USAFDfs)):
  USAFDfs[i].pop("Unnamed: 0");
  USAFDfs[i].pop('OBS');
  USAFDfs[i].pop('Timetag');

Concatenating dataframes on top of each other.

In [11]:
for i in range(len(USAFDfs)):
  df = pd.concat([df, USAFDfs[i]], ignore_index=True)

In [15]:
# Data frame with columns sorted by ARNUMBER so that we can easily visualize the changes of each AR
DF = df.sort_values(by=['ARNUMBER', 'LOND'], ignore_index=True)
SmallDf = DF#.head(30) #Using 10 elements of original data
print("Number of entries is:",len(DF))


Number of entries is: 66361


Every column of small Df is turned into a series, then into a list.

In [16]:
 #Separating the columns into list 
    
Obs_ID = SmallDf['OBS_ID'].squeeze()
Obs_ID_List = Obs_ID.tolist()
Timetag = SmallDf['TIMETAG'].squeeze()
Timetag_List = Timetag.tolist()
ARNumber = SmallDf['ARNUMBER'].squeeze()
ARNumber_List = ARNumber.tolist()
Loc = SmallDf['LOC'].squeeze()
Loc_List = Loc.tolist()
Latd = SmallDf['LATD'].squeeze()
Latd_List = Latd.tolist()
Lond = SmallDf['LOND'].squeeze()
Lond_List = Lond.tolist()
Lonh = SmallDf['LONH'].squeeze()
Lonh_List = Lonh.tolist()
Area = SmallDf['AREA'].squeeze()
Area_List = Area.tolist()
Z = SmallDf['Z'].squeeze()
Z_List = Z.tolist()
LL = SmallDf['LL'].squeeze()
LL_List = LL.tolist()
NN = SmallDf['NN'].squeeze()
NN_List = NN.tolist()
MagType = SmallDf['MAGTYPE'].squeeze()
MagType_List = MagType.tolist()
Type = [0] #empty list to store type "Real (0) or Prediciton (1)"

Next, We extrapolate locations of Active Regions using Carrington Rate and we extend the active region ONLY if this is the last record for this active region in the entire series
(i.e. only if it disappears the next day and does not appear anymore). The list "Type" stores 0 for non-extraploated and 1 for extrapolated entries.

In [17]:

j = 0
while(True):
    if(j>=len(Lond_List)): break
    #if((Lond_List[j]+14.2) <= 270.0): #If current pos. does not surpass 270 when +14.2, then insert a new
    #   #a new with +14.2 
    # ---------------------------------------------------------------
    # Let's extend the active region ONLY if this is the last record for this active region in the entire series
    # (i.e. only if it disappears the next day and does not appear anymore).
    # This leads to a slightly modified cycle condition:
    # if (longitude+14.2 < 270) AND (no ARs with similar numbers presented further)
    # ---------------------------------------------------------------
    if ( ((Lond_List[j]+14.2) <= 270.0) and (ARNumber_List[j] not in ARNumber_List[j+1:]) ):
        Lond_List.insert(j+1, (Lond_List[j]+14.2))
        ARNumber_List.insert(j+1, ARNumber_List[j])
        Obs_ID_List.insert(j+1, Obs_ID_List[j])
        Latd_List.insert(j+1, Latd_List[j])
        Lonh_List.insert(j+1, Lonh_List[j])
        Area_List.insert(j+1, Area_List[j])
        Z_List.insert(j+1, Z_List[j])
        LL_List.insert(j+1, LL_List[j])
        NN_List.insert(j+1, NN_List[j])
        MagType_List.insert(j+1, MagType_List[j])
        Timetag_List.insert(j+1, Timetag_List[j]) #Then, convert to DF timestamp and iterate to apply DateOffSet
        Loc_List.insert(j+1, Loc_List[j])
        
        #Missing Timestags and Location
        
        Type.insert(j+1, 1)
        j+=1
        
    else:
        if((j+1) < len(Lond_List)): #So that it will not go past the the number of total elements
            Type.insert(j+1, 0) #Type list is empty, so I need to add 0's too.
            #MagType_List
            
            
            #ARNumber_List.insert(j+1, ARNumber_List[j+1]) --> Don't need because there already different
               ##.  AR numbers for the real data. So, a real data is identified just iterate to next element
                ##   and begin repeating AR numbers then.
        j+=1
        
        
#Lond_List

The above loop did not update the dates but only the locating if the 14.2 degrees were added. Thus, we need to add an extra day for every 14.2 degrees as required by the Carrington rate.

In [18]:
DFdate = pd.DataFrame({'Timetag': Timetag_List, 'Type': Type})
DFdate['Timetag'] = pd.to_datetime(DFdate['Timetag'])

for index, row in DFdate.iterrows():
    if (row['Type'] == 0):
        dt = 0 
        continue
    else:
        dt+=1
        if dt == 1:
            txt = "{} day".format(dt)
            
        else: 
            txt = "{} days".format(dt)
            
            
    DFdate.iloc[index, 0] = row['Timetag'] + pd.Timedelta(txt)#+ pd.Timedelta(txt)

DFdate.head(50) #50 entries of the new updated dates
Updated_Timetag_Series = DFdate['Timetag']
#Updated_Timetag_Series.dt.date
Updated_Timetag_List = Updated_Timetag_Series.tolist()
Updated_Timetag_List[10] #List of updated dates

Timestamp('1982-04-24 00:00:00')

The McIntosh classes that appear in upper case are turned to lower cases for the second and third components and putting. Then, they are stored in a list.

In [37]:
Class = ['A', 'B', 'C', 'D', 'E', 'F', 'H']
LowPen = ['a', 'h', 'k', 'r', 's', 'x']  #Lower case Penumbra
CapPen = ['A','H','K','R','S','X']
LowDist = ['i', 'c', 'o', 'x']
CapDist = ['I','C','O','X']
z = []
for i in range (len(Z_List)):
    z.append(list(Z_List[i])) #Z_List contains strings. list() converts to list'z' is a tuple. I have list with len(Z_List) and 3 columns
for i in range(len(z)):
    for j in range(len(LowPen)):
        if z[i][1] == CapPen[j]:
            z[i][1] = LowPen[j]
            break
            
for i in range(len(z)):
    for j in range(len(LowDist)):
        if z[i][2] == CapDist[j]:
            z[i][2] = LowDist[j]
            break

The following code overwrites the locations. Note that this list will not be important when performing our statistical study.

In [38]:
#overwrites Loc_List

# Adds the location North, South, East, and West accordingly.

# ---------> ISSUE: At this point, I am adding 90+ degrees as West. [90, 180] should be West too, but
##.      [180, 270] should East, if I am correct. I tried using 'or' inside the if statement to fix it,
##.      but it seems like it does not do its job. I can certainly fix it, if I keep playing with it.


Lond_List = [round(x) for x in Lond_List] #convert to integer
for i in range(len(Lond_List)):
    if (Latd_List[i] < 0 and Lond_List[i] < 0):
        loc = "S{}E{}".format(Latd_List[i]*-1, Lond_List[i]*-1)
    elif (Latd_List[i] < 0 and Lond_List[i] > 0):
        loc = "S{}W{}".format(Latd_List[i]*-1, Lond_List[i])
    elif (Latd_List[i] > 0 and (Lond_List[i] < 0 or Lond_List[i] > 180) ):
        loc = "N{}E{}".format(Latd_List[i], -1*Lond_List[i])
    elif (Latd_List[i] > 0 and (Lond_List[i] > 0 or Lond_List[i] < 180)):
        loc = "N{}W{}".format(Latd_List[i], Lond_List[i])
        
    Loc_List[i] = loc

In [39]:
Dict = {'OBS_ID': Obs_ID_List, 'TIMETAG': Updated_Timetag_List, 'ARNUMBER': ARNumber_List, 'LOC': Loc_List, \
       'LATD': Latd_List, 'LOND': Lond_List, 'LONH': Lonh_List, 'AREA': Area_List, 'Z': Z_List, \
        'z': z,'LL': LL_List, 'NN': NN_List, 'MAGTYPE': MagType_List, 'REAL(0)/PRED(1)': Type}

Database = pd.DataFrame(Dict)

In our "Database", we need to rewrite McIntosh components to keep all McIntosh records consistent.

In [40]:
print(Database.Z.unique())

['AXX' 'BXO' 'FKI' 'FHI' 'EKO' 'EHO' 'EHI' 'CHO' 'CSO' 'DAI' 'EAI' 'DSO'
 'HHX' 'DAO' 'HSX' 'CAO' 'DHO' 'DSI' 'EKI' 'EKC' 'DKI' 'DRO' 'DKO' 'EAO'
 'HKX' 'HAX' 'HRX' 'CKO' 'DHI' 'DKC' 'CKI' 'CRO' 'DRI' 'CHI' 'ESO' 'CSI'
 'FKC' 'FAO' 'CAI' 'CRI' 'FHO' 'FSO' 'ESI' 'EAC' 'FSI' 'FAI' 'EHC' 'DHC'
 'ERO' 'FKO' 'DSC' 'ERI' 'FRO' 'DAC' 'FAC' 'ESC' 'FSC' 'FHC' 'BXI' 'Hrx'
 'Axx' 'Hsx' 'Hax' 'Eki' 'Esi' 'Eai' 'Eso' 'Eao' 'Fhi' 'Cao' 'Cso' 'Dso'
 'Dao' 'Dai' 'Cro' 'Bxo' 'Dsi' 'Fso' 'Dko' 'Dki' 'Dro' 'Fao' 'Eko' 'Fac'
 'Fai' 'Fki' 'Eho' 'Fkc' 'Hkx' 'Cho' 'Dho' 'Dac' 'Fsi' 'Ekc' 'Fho' 'Cko'
 'Cai' 'Csi' 'Fko' 'Eac' 'Dkc' 'Hhx' 'Ehi' 'Dri' 'Dhi' 'Fro' 'Chi' 'Dhc'
 'Ckc' 'Esc' 'Hxx' 'Dsc' 'Bxi' 'Fsc' 'Ero' 'Fhc' 'Ehc' 'Cki' 'Drc' 'Cri'
 'Hso' 'Eri']


In [41]:
Database.loc[Database['Z'] == 'Hrx', 'Z'] = "HRX"
Database.loc[Database['Z'] == 'Axx', 'Z'] = "AXX"
Database.loc[Database['Z'] == 'Hsx', 'Z'] = "HSX"
Database.loc[Database['Z'] == 'Hax', 'Z'] = "HAX"
Database.loc[Database['Z'] == 'Eki', 'Z'] = "EKI"
Database.loc[Database['Z'] == 'Esi', 'Z'] = "ESI"
Database.loc[Database['Z'] == 'Eai', 'Z'] = "EAI"
Database.loc[Database['Z'] == 'Eso', 'Z'] = "ESO"
Database.loc[Database['Z'] == 'Eao', 'Z'] = "EAO"
Database.loc[Database['Z'] == 'Fhi', 'Z'] = "FHI"
Database.loc[Database['Z'] == 'Cao', 'Z'] = "CAO"
Database.loc[Database['Z'] == 'Cso', 'Z'] = "CSO"
Database.loc[Database['Z'] == 'Dso', 'Z'] = "DSO"
Database.loc[Database['Z'] == 'Dao', 'Z'] = "DAO"
Database.loc[Database['Z'] == 'Dai', 'Z'] = "DAI"
Database.loc[Database['Z'] == 'Cro', 'Z'] = "CRO"
Database.loc[Database['Z'] == 'Bxo', 'Z'] = "BXO"
Database.loc[Database['Z'] == 'Dsi', 'Z'] = "DSI"
Database.loc[Database['Z'] == 'Fso', 'Z'] = "FSO"
Database.loc[Database['Z'] == 'Dko', 'Z'] = "DKO"
Database.loc[Database['Z'] == 'Dki', 'Z'] = "DKI"
Database.loc[Database['Z'] == 'Dro', 'Z'] = "DRO"
Database.loc[Database['Z'] == 'Fao', 'Z'] = "FAO"
Database.loc[Database['Z'] == 'Eko', 'Z'] = "EKO"
Database.loc[Database['Z'] == 'Fac', 'Z'] = "FAC"
Database.loc[Database['Z'] == 'Fai', 'Z'] = "FAI"
Database.loc[Database['Z'] == 'Fki', 'Z'] = "FKI"
Database.loc[Database['Z'] == 'Eho', 'Z'] = "EHO"
Database.loc[Database['Z'] == 'Fkc', 'Z'] = "FKC"
Database.loc[Database['Z'] == 'Hkx', 'Z'] = "HKX"
Database.loc[Database['Z'] == 'Cho', 'Z'] = "CHO"
Database.loc[Database['Z'] == 'Dho', 'Z'] = "DHO"
Database.loc[Database['Z'] == 'Dac', 'Z'] = "DAC"
Database.loc[Database['Z'] == 'Fsi', 'Z'] = "FSI"
Database.loc[Database['Z'] == 'Ekc', 'Z'] = "EKC"
Database.loc[Database['Z'] == 'Fho', 'Z'] = "FHO"
Database.loc[Database['Z'] == 'Cko', 'Z'] = "CKO"
Database.loc[Database['Z'] == 'Cai', 'Z'] = "CAI"
Database.loc[Database['Z'] == 'Csi', 'Z'] = "CSI"
Database.loc[Database['Z'] == 'Fko', 'Z'] = "FKO"
Database.loc[Database['Z'] == 'Eac', 'Z'] = "EAC"
Database.loc[Database['Z'] == 'Dkc', 'Z'] = "DKC"
Database.loc[Database['Z'] == 'Hhx', 'Z'] = "HHX"
Database.loc[Database['Z'] == 'Ehi', 'Z'] = "EHI"
Database.loc[Database['Z'] == 'Dri', 'Z'] = "DRI"
Database.loc[Database['Z'] == 'Dhi', 'Z'] = "DHI"
Database.loc[Database['Z'] == 'Fro', 'Z'] = "FRO"
Database.loc[Database['Z'] == 'Chi', 'Z'] = "CHI"
Database.loc[Database['Z'] == 'Dhc', 'Z'] = "DHC"
Database.loc[Database['Z'] == 'Ckc', 'Z'] = "CKC"
Database.loc[Database['Z'] == 'Esc', 'Z'] = "ESC"
Database.loc[Database['Z'] == 'Hxx', 'Z'] = "HXX"
Database.loc[Database['Z'] == 'Dsc', 'Z'] = "DSC"
Database.loc[Database['Z'] == 'Bxi', 'Z'] = "BXI"
Database.loc[Database['Z'] == 'Fsc', 'Z'] = "FSC"
Database.loc[Database['Z'] == 'Ero', 'Z'] = "ERO"
Database.loc[Database['Z'] == 'Fhc', 'Z'] = "FHC"
Database.loc[Database['Z'] == 'Ehc', 'Z'] = "EHC"
Database.loc[Database['Z'] == 'Cki', 'Z'] = "CKI"
Database.loc[Database['Z'] == 'Drc', 'Z'] = "DRC"
Database.loc[Database['Z'] == 'Cri', 'Z'] = "CRI"
Database.loc[Database['Z'] == 'Hso', 'Z'] = "HSO"
Database.loc[Database['Z'] == 'Eri', 'Z'] = "ERI"

Note below how they are all now consistent on column 'Z,' i.e. all were turned into capital case.

In [42]:
print(Database.Z.unique())

['AXX' 'BXO' 'FKI' 'FHI' 'EKO' 'EHO' 'EHI' 'CHO' 'CSO' 'DAI' 'EAI' 'DSO'
 'HHX' 'DAO' 'HSX' 'CAO' 'DHO' 'DSI' 'EKI' 'EKC' 'DKI' 'DRO' 'DKO' 'EAO'
 'HKX' 'HAX' 'HRX' 'CKO' 'DHI' 'DKC' 'CKI' 'CRO' 'DRI' 'CHI' 'ESO' 'CSI'
 'FKC' 'FAO' 'CAI' 'CRI' 'FHO' 'FSO' 'ESI' 'EAC' 'FSI' 'FAI' 'EHC' 'DHC'
 'ERO' 'FKO' 'DSC' 'ERI' 'FRO' 'DAC' 'FAC' 'ESC' 'FSC' 'FHC' 'BXI' 'CKC'
 'HXX' 'DRC' 'HSO']


Next, we need to normalize all values for the purpose of machine learning applications.

In [43]:
MaxArea = Database.AREA.max()
NORMAREA = Database['AREA'] / MaxArea

In [44]:
MaxLond = Database.LOND.max()
NORMLOND = Database['LOND'] / MaxLond

Inserting Normalized Area and Longitude into database (ONLY needed to be run once).

In [45]:
Database.insert(8, "NORMAREA", NORMAREA)
Database.insert(6, "NORMLOND", NORMLOND)

In [48]:
ttag = Database.TIMETAG.squeeze()

In [52]:
Database['TIMETAG'] = Database['TIMETAG'].dt.strftime('%Y-''%m-''%d ' '%r') #Date and time to string

The line below produces the json file used in our study.

In [54]:
Database.to_json("Datasets/ARDatabase.json")